In [ ]:
!pip install pandas
%pip install statsmodels
%pip install sqlalchemy
%pip install matplotlib
%pip install sklearn
! conda install -c conda-forge xgboost
import pandas as pd
import statsmodels
import scipy as sc
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import urllib
from matplotlib import pylab as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor, plot_importance

In [ ]:
#Выгрузка данных из базы
from sqlalchemy import create_engine
import urllib
server = 'test' 
database = 'test_base' 
username = 'test_username' 
password = 'test_password' 
params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                           "SERVER= "+server+";"
                           "DATABASE="+database+";"
                           "UID="+username+";"
                           "PWD="+password+";"
                          )
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
df = pd.read_sql("SELECT * FROM table_test1 order by date", engine)
df_test = pd.read_sql("SELECT * FROM table_test2 order by date", engine)
engine.dispose()

In [ ]:
#Преобразование текстовых данных в числовые для данных level3
label_level3 = LabelEncoder()
label_level3.fit(df['level3'])
df['level3_lbl'] = label_level3.transform(df['level3'])
df_test['level3_lbl'] = label_level3.transform(df_test['level3'])

#Преобразование текстовых данных в числовые для данных level5
label_level5 = LabelEncoder()
label_level5.fit(df['level5'])
df['level5_lbl'] = label_level5.transform(df['level5'])
df_test['level5_lbl'] = label_level5.transform(df_test['level5'])

#Преобразование текстовых данных в числовые для данных level4
label_level4= LabelEncoder()
label_level4.fit(df['level4'])
df['level4_lbl'] = label_level4.transform(df['level4'])
df_test['level4_lbl'] = label_level4.transform(df_test['level4'])

In [ ]:
#Удаление пустых строк в тренировочной и тестовой выборках
Train = df.dropna()
Test = df_test.dropna()

X_forecast = Test[:0]
#Получаю уникальные значения level5
level5_variants = Train.level5.unique()

In [ ]:
for level5 in level5_variants:
    X_train = Train[Train['level5']== level5].copy()
    X_test = Test[Test['level5']== level5].copy()
    Y_train = X_train['qty_all'].copy()

    #Подготовка массивов
    X_train_v = X_train[[
    'avg', 
    'd_avg_price'
    ]]
    X_test_v = X_test[[ 
    'avg', 
    'd_avg_price'
    ]]
    
    X_test_k = X_test[['level3_lbl','level4_lbl','level5_lbl','Shop','holiday','dw','m']].astype(str)
    X_train_k = X_train[['level3_lbl','level4_lbl','level5_lbl','Shop','holiday','dw','m']].astype(str)

    #Cтандартизируем диапазон функциональных возможностей входного набора данных
    scaler = StandardScaler()
    scaler.fit(X_train_v)

    X_train_v_sc = scaler.transform(X_train_v)
    X_test_v_sc = scaler.transform(X_test_v)
    #Еще преобразование данных для того, чтобы модель не запуталась,
    #ложно предположив, что данные связаны порядком или иерархией, которого на самом деле нет
    one = OneHotEncoder(sparse = False)
    one.fit (X_train_k)
    X_train_k_d = one.transform(X_train_k)
    X_test_k_d = one.transform(X_test_k)
    
    #увеличение массива данных по горизонтали
    X_train_all = np.hstack((X_train_v_sc, X_train_k_d))
    X_test_all = np.hstack((X_test_v_sc, X_test_k_d))
    
    #опытным путем подбираем параметры для моделей
    params = {
              'min_child_weight':[4, 6, 8],
              'learning_rate': [0.1],
              'n_estimators': [40,50,60,80,100],
              'max_depth': [ 4, 5, 6]
             }
    #Используем модель основанную на деревьях решения с методами градиентного бустинга
    model = XGBRegressor (random_state=0)
    #Используем хелпера, который подберет наилучшие параметры для модели
    grid_search = GridSearchCV (
                       estimator = model, 
                       param_grid = params,
                       scoring = 'neg_mean_absolute_error', 
                       verbose = 1 )
    grid_search.fit(X_train_all, Y_train) 
    #Прогнозирование
    Y_pred = grid_search.predict(X_test_all)
    #Заливка данных в столбец прогноза
    X_test['forecast'] = Y_pred
    X_forecast = pd.concat([X_forecast, X_test])
#Заливка данных в базу
X_forecast.to_sql('table_result', con = engine, if_exists = 'replace', chunksize = 1000)